In [2]:
!pip install mediapipe

     |████████████████████████████████| 35.2 MB 15.4 MB/s eta 0:00:01
     |████████████████████████████████| 63.1 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 10.7 MB/s eta 0:00:01


In [1]:
import cv2
import mediapipe as mp
import time

In [15]:
cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands() # なんかこの.Hands()の使い方解説してた
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    # print(results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                print(id, cx, cy)
                # if id == 0:
                cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED) # imgの中、(cx, cy)の場所に大きさ10で色を(255, 0, 255)で表示。cv2.FILLEDは塗りつぶし？
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, 
                (255, 0, 255), 3)
    
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)
    # cv2.destroyAllWindows()
  


0 1544 953
1 1396 889
2 1334 759
3 1326 618
4 1321 516
5 1458 680
6 1431 477
7 1417 389
8 1406 336
9 1568 677
10 1584 455
11 1573 355
12 1557 299
13 1650 695
14 1676 497
15 1662 405
16 1645 352
17 1701 726
18 1722 578
19 1716 501
20 1703 454
0 1543 864
1 1402 780
2 1332 661
3 1315 549
4 1300 458
5 1428 548
6 1400 401
7 1381 321
8 1365 258
9 1519 533
10 1513 372
11 1497 280
12 1480 214
13 1594 541
14 1592 390
15 1582 308
16 1567 246
17 1657 564
18 1669 448
19 1666 376
20 1656 317
0 1528 812
1 1385 730
2 1312 627
3 1293 531
4 1277 449
5 1401 494
6 1373 378
7 1360 313
8 1347 251
9 1483 475
10 1478 344
11 1468 266
12 1455 201
13 1550 484
14 1561 365
15 1553 295
16 1542 234
17 1622 517
18 1643 421
19 1642 363
20 1638 305
0 1498 767
1 1371 712
2 1307 618
3 1297 524
4 1278 450
5 1401 481
6 1374 373
7 1357 308
8 1345 247
9 1482 466
10 1480 339
11 1471 261
12 1465 192
13 1552 479
14 1561 363
15 1556 293
16 1547 227
17 1620 515
18 1646 425
19 1652 367
20 1655 307
0 1499 751
1 1381 707
2 1317 616

KeyboardInterrupt: 

In [26]:
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0, trackCon=0.5): 
        """
        mode:手の検出モード。Falseは単一のみ。Trueは複数
        maxHands:検出する手の最大数
        detectionCon:手を検出するための最低信頼度。動画では0.5だったが、エラー。0だと動く
        trackCon:手をトラックするための最低信頼度
        """
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon) # なんかこの.Hands()の使い方解説してた
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self, img, draw=True):
        """
        手を見つける
        """
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB) # 手の検出。手の位置や特徴点などをresultsに格納。手がない時はNone。hands()は動画内で説明あり
        
        if self.results.multi_hand_landmarks: # 画像に手が検出できた時
            for handLms in self.results.multi_hand_landmarks:
                if draw: # 手の指の位置に円を描画。draw=Falseで描画しない
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS) # HAND_CONNECTIONSで点を結ぶ線を描画
        
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
        """
        検出した手の指の位置をリストで返す
        """
        lmList = []
        if self.results.multi_hand_landmarks: # 手を検出できた時
            myHand = self.results.multi_hand_landmarks[handNo] # 最初に検出された手を代入
            
            for id, lm in enumerate(myHand.landmark):
                    # 位置を計算
                    h, w, c = img.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    # それぞれの点の場所と位置を追加
                    lmList.append([id, cx, cy])
                    if draw: # draw=Falseで円を表示しないことも可能
                        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED) # imgの中、(cx, cy)の場所に大きさ10で色を(255, 0, 255)で表示。cv2.FILLEDは塗りつぶし？
                
        return lmList


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0) # 0はカメラのid
    detector = handDetector()

    while True:
        success, img = cap.read() # カメラからの入力を受け取る
        img = detector.findHands(img) 
        lmList = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])

        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, 
                    (255, 0, 255), 3)
        
        
        cv2.imshow("Image", img)
        cv2.waitKey(1)


if __name__ == '__main__':
    main()

[4, 1177, 772]
[4, 1156, 653]
[4, 1134, 559]
[4, 1137, 522]
[4, 1134, 496]
[4, 1119, 494]
[4, 1106, 505]
[4, 1099, 513]
[4, 1099, 521]
[4, 1103, 529]
[4, 1110, 538]
[4, 1113, 550]
[4, 1117, 557]
[4, 1119, 567]
[4, 1120, 571]
[4, 1124, 570]
[4, 1123, 568]
[4, 1124, 572]
[4, 1127, 573]
[4, 1130, 575]
[4, 1134, 570]
[4, 1138, 568]
[4, 1133, 566]
[4, 1125, 566]
[4, 1119, 563]
[4, 363, 559]
[4, 388, 530]
[4, 368, 533]
[4, 336, 554]
[4, 312, 564]
[4, 302, 570]
[4, 279, 565]
[4, 272, 569]
[4, 269, 574]
[4, 268, 577]
[4, 265, 578]
[4, 256, 579]
[4, 263, 582]
[4, 282, 588]
[4, 316, 594]
[4, 377, 600]
[4, 462, 593]
[4, 519, 591]
[4, 561, 586]
[4, 589, 577]
[4, 600, 571]
[4, 603, 573]
[4, 587, 571]
[4, 558, 568]
[4, 501, 573]
[4, 437, 596]
[4, 374, 661]
[4, 1192, 554]
[4, 1185, 552]
[4, 1178, 549]
[4, 1178, 549]
[4, 1177, 553]
[4, 1184, 549]
[4, 1196, 541]
[4, 1188, 535]
[4, 1150, 567]
[4, 1177, 588]
[4, 1319, 594]
[4, 1350, 604]
[4, 1353, 609]
[4, 1354, 617]
[4, 1367, 617]
[4, 1387, 617]
[4, 145

KeyboardInterrupt: 

In [ ]:
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0) # 0はカメラのid
detector = handDetector()

while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img)
    if len(lmList) != 0:
        print(lmList[4])

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, 
                (255, 0, 255), 3)
    
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)